In [ ]:
#pip install pandas requests tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
new_directory = "/content/drive/MyDrive/NASA_SPACE_APP/"
os.chdir(new_directory)
print(f"current working directory: {os.getcwd()}")

current working directory: /content/drive/MyDrive/NASA_SPACE_APP


In [ ]:
import os
import zipfile
import pandas as pd
import requests
from tqdm import tqdm
import numpy as np

# Config
target_years = [2023, 2024, 2025]
OIL_CSV = "/content/drive/MyDrive/NASA_SPACE_APP/candidates_2015_2025.csv"
DAILY_STATS_CSV = "/content/drive/MyDrive/NASA_SPACE_APP/AIS_daily_stats_23_24_25.csv"
OIL_POINT_CSV = "/content/drive/MyDrive/NASA_SPACE_APP/oil_points_AIS_stats_23_24_25.csv"
#AIS_BASE_URL = "https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2024/"
TEMP_DIR = "/content/temp_ais/"
BUFFER_DEG = 0.05  # ~5 km

os.makedirs(TEMP_DIR, exist_ok=True)
print("Temp directory prepared.")

print("Loading oil candidate points...")
oil_df = pd.read_csv(OIL_CSV)
oil_df['date_dt'] = pd.to_datetime(oil_df['date'])
oil_df = oil_df[oil_df['date_dt'].dt.year.isin(target_years)]
oil_df['date_fmt'] = pd.to_datetime(oil_df['date']).dt.strftime('%Y_%m_%d')
print(f"Loaded {oil_df.shape[0]} oil candidate detections.")

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

daily_stats_list = []
oil_points_list = []

dates = oil_df['date_fmt'].unique()
print(f"{len(dates)} AIS dates to process: {dates}")

for date_ in tqdm(dates, desc="Processing AIS dates"):
    print(f"\n--- AIS Date: {date_} ---")
    date_str = date_.replace("-", "_")  # convert just once
    year_str = date_str[:4]
    AIS_BASE_URL = f"https://coast.noaa.gov/htdata/CMSP/AISDataHandler/{year_str}/"
    zip_name = f"AIS_{date_str}.zip"
    zip_path = os.path.join(TEMP_DIR, zip_name)
    csv_name = f"AIS_{date_str}.csv"
    csv_path = os.path.join(TEMP_DIR, csv_name)
    url = AIS_BASE_URL + zip_name

    print("Step 1: Downloading AIS ZIP...")
    try:
        resp = requests.get(url, stream=True, timeout=60)
        resp.raise_for_status()
        with open(zip_path, 'wb') as f:
            for chunk in resp.iter_content(1024*1024):
                f.write(chunk)
        print("Step 2: Extracting AIS ZIP...")
        with zipfile.ZipFile(zip_path, 'r') as z:
            z.extractall(TEMP_DIR)
        print(f"Extracted AIS CSV: {csv_name}")

        print("Step 3: Loading AIS DataFrame...")
        ais_df = pd.read_csv(csv_path, low_memory=False)
        print(f"AIS DataFrame shape: {ais_df.shape}")

    except Exception as e:
        print(f"Error processing {date_}: {e}")
        continue
    finally:
        if os.path.exists(zip_path):
            os.remove(zip_path)
        print("ZIP file cleanup done.")

    print("Step 4: Calculating daily AIS stats...")
    daily_stats = {
        'Date': pd.to_datetime(date_str, format='%Y_%m_%d').strftime('%Y-%m-%d'),
        'Unique_Vessels': ais_df['MMSI'].nunique(),
        'Total_Positions': len(ais_df),
        'Unique_Lat_Lon_Count': ais_df[['LAT','LON']].drop_duplicates().shape[0],
        'Min_LAT': ais_df['LAT'].min(),
        'Max_LAT': ais_df['LAT'].max(),
        'Min_LON': ais_df['LON'].min(),
        'Max_LON': ais_df['LON'].max()
    }
    print("Daily AIS stats:", daily_stats)
    daily_stats_list.append(daily_stats)

    print("Step 5: Analyzing oil candidates for this date...")
    points_today = oil_df[oil_df['date_fmt']==date_]
    print(f"Found {len(points_today)} candidates.")

    for idx, row in points_today.iterrows():
        near_ships = ais_df[
            (ais_df['LAT'] >= row['latitude'] - BUFFER_DEG) &
            (ais_df['LAT'] <= row['latitude'] + BUFFER_DEG) &
            (ais_df['LON'] >= row['longitude'] - BUFFER_DEG) &
            (ais_df['LON'] <= row['longitude'] + BUFFER_DEG)
        ]

        num_ships = near_ships['MMSI'].nunique()
        if len(near_ships) > 0:
            closest_distance_km = haversine(row['latitude'], row['longitude'],
                                            near_ships['LAT'], near_ships['LON']).min()
            avg_speed = near_ships['SOG'].mean()
            vessel_type_counts = near_ships['VesselType'].value_counts().to_dict()
        else:
            closest_distance_km = None
            avg_speed = None
            vessel_type_counts = {}

        print(f"Oil point {idx}: lat={row['latitude']}, lon={row['longitude']} | Ships nearby={num_ships} | Closest(km)={closest_distance_km} | Avg speed={avg_speed}")

        res = {
            'Date': pd.to_datetime(date_str, format='%Y_%m_%d').strftime('%Y-%m-%d'),
            'oil_lat': row['latitude'],
            'oil_lon': row['longitude'],
            'num_ships_near_point': num_ships,
            'closest_ship_distance_km': closest_distance_km,
            'avg_ship_speed': avg_speed
        }
        for vtype, count in vessel_type_counts.items():
            res[f'vessel_type_{vtype}'] = count

        oil_points_list.append(res)

    if os.path.exists(csv_path):
        os.remove(csv_path)
        print(f"CSV cleanup for {csv_name} done.")

print("\nSaving all outputs ...")
pd.DataFrame(daily_stats_list).to_csv(DAILY_STATS_CSV, index=False)
pd.DataFrame(oil_points_list).to_csv(OIL_POINT_CSV, index=False)
print(f"Saved daily AIS stats to: {DAILY_STATS_CSV}")
print(f"Saved oil-point AIS stats to: {OIL_POINT_CSV}")


Temp directory prepared.
Loading oil candidate points...
Loaded 1991 oil candidate detections.
159 AIS dates to process: ['2023_01_03' '2023_01_10' '2023_01_15' '2023_01_22' '2023_01_27'
 '2023_02_08' '2023_02_15' '2023_02_20' '2023_03_04' '2023_03_11'
 '2023_03_16' '2023_03_23' '2023_03_28' '2023_04_04' '2023_04_09'
 '2023_04_16' '2023_04_21' '2023_05_03' '2023_05_10' '2023_05_15'
 '2023_05_22' '2023_05_27' '2023_06_08' '2023_06_15' '2023_06_20'
 '2023_06_27' '2023_07_09' '2023_07_14' '2023_07_21' '2023_08_07'
 '2023_08_19' '2023_08_26' '2023_08_31' '2023_09_07' '2023_09_12'
 '2023_09_19' '2023_09_24' '2023_10_01' '2023_10_06' '2023_10_13'
 '2023_10_18' '2023_10_25' '2023_10_30' '2023_11_06' '2023_11_11'
 '2023_11_18' '2023_11_23' '2023_11_30' '2023_12_05' '2023_12_12'
 '2023_12_17' '2023_12_24' '2023_12_29' '2024_01_05' '2024_01_10'
 '2024_01_17' '2024_01_22' '2024_01_29' '2024_02_03' '2024_02_10'
 '2024_02_15' '2024_02_22' '2024_02_27' '2024_03_05' '2024_03_10'
 '2024_03_17' '2024_0

Processing AIS dates:   0%|          | 0/159 [00:00<?, ?it/s]


--- AIS Date: 2023_01_03 ---
Step 1: Downloading AIS ZIP...
Step 2: Extracting AIS ZIP...
Extracted AIS CSV: AIS_2023_01_03.csv
Step 3: Loading AIS DataFrame...
AIS DataFrame shape: (7764316, 17)
ZIP file cleanup done.
Step 4: Calculating daily AIS stats...
Daily AIS stats: {'Date': '2023-01-03', 'Unique_Vessels': 15949, 'Total_Positions': 7764316, 'Unique_Lat_Lon_Count': 3328499, 'Min_LAT': np.float64(-4.59537), 'Max_LAT': np.float64(58.00593), 'Min_LON': np.float64(-177.23195), 'Max_LON': np.float64(166.85462)}
Step 5: Analyzing oil candidates for this date...
Found 16 candidates.
Oil point 4670: lat=37.07854447316908, lon=-76.33926388459538 | Ships nearby=2 | Closest(km)=0.7900638760750851 | Avg speed=0.43832020997375326
Oil point 4671: lat=39.68334252710496, lon=-76.19965432641052 | Ships nearby=0 | Closest(km)=None | Avg speed=None
Oil point 4672: lat=37.32722945812118, lon=-76.8763935436474 | Ships nearby=0 | Closest(km)=None | Avg speed=None
Oil point 4673: lat=39.2137849791820

Processing AIS dates:   1%|          | 1/159 [00:26<1:09:35, 26.43s/it]

Oil point 4681: lat=37.5492397986249, lon=-75.86663133426741 | Ships nearby=0 | Closest(km)=None | Avg speed=None
Oil point 4682: lat=36.94709711009913, lon=-76.04567074652736 | Ships nearby=48 | Closest(km)=0.24218550795250363 | Avg speed=2.4025272200579364
Oil point 4683: lat=37.158555875725824, lon=-76.65732967341695 | Ships nearby=11 | Closest(km)=1.0957482425218878 | Avg speed=2.3760082730093077
Oil point 4684: lat=39.38341433265008, lon=-76.39936747439695 | Ships nearby=0 | Closest(km)=None | Avg speed=None
Oil point 4685: lat=37.64830399387009, lon=-76.5517764640962 | Ships nearby=3 | Closest(km)=0.732081078856547 | Avg speed=2.434533898305085
CSV cleanup for AIS_2023_01_03.csv done.

--- AIS Date: 2023_01_10 ---
Step 1: Downloading AIS ZIP...
Step 2: Extracting AIS ZIP...
Extracted AIS CSV: AIS_2023_01_10.csv
Step 3: Loading AIS DataFrame...
AIS DataFrame shape: (8135468, 17)
ZIP file cleanup done.
Step 4: Calculating daily AIS stats...
Daily AIS stats: {'Date': '2023-01-10', '

...

Processing AIS dates:  99%|█████████▉| 158/159 [1:04:04<00:00,  4.49it/s]

Error processing 2025_09_20: 404 Client Error: The requested content does not exist. for url: https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2025/AIS_2025_09_20.zip
ZIP file cleanup done.

--- AIS Date: 2025_09_25 ---
Step 1: Downloading AIS ZIP...


Processing AIS dates: 100%|██████████| 159/159 [1:04:04<00:00, 24.18s/it]

Error processing 2025_09_25: 404 Client Error: The requested content does not exist. for url: https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2025/AIS_2025_09_25.zip
ZIP file cleanup done.

Saving all outputs ...


Saved daily AIS stats to: /content/drive/MyDrive/NASA_SPACE_APP/AIS_daily_stats_23_24_25.csv
Saved oil-point AIS stats to: /content/drive/MyDrive/NASA_SPACE_APP/oil_points_AIS_stats_23_24_25.csv


In [ ]:
import pandas as pd
import re

# === 1️⃣ File path ===
INPUT_CSV = "/content/drive/MyDrive/NASA_SPACE_APP/oil_points_AIS_stats_23_24_25.csv"
OUTPUT_CSV = "/content/drive/MyDrive/NASA_SPACE_APP/oil_points_AIS_stats_renamed_18_19_20_.csv"

# === 2️⃣ Vessel Type Mapping ===
vessel_type_map = {
    range(20, 30): "Wing_in_ground_WIG_hovercraft_hydrofoils",
    30: "Fishing",
    31: "Towing",
    32: "Towing_length_gt_200m_or_width_gt_25m",
    33: "Dredging_or_underwater_ops",
    34: "Diving_ops",
    35: "Military_operations",
    36: "Sailing",
    37: "Pleasure_craft_yachts",
    40: "High_speed_craft_HSC",
    50: "Pilot_vessel",
    51: "Search_and_rescue_vessel",
    52: "Tug",
    53: "Port_tender",
    54: "Anti_pollution_equipment",
    55: "Law_enforcement_vessel",
    56: "Spare_local_use",
    57: "Spare_local_use",
    58: "Medical_transport",
    59: "Noncombatant_ship_military_support",
    60: "Passenger",
    69: "Passenger_hazardous_cargo",
    70: "Cargo",
    71: "Cargo_hazardous_A",
    72: "Cargo_hazardous_B",
    73: "Cargo_hazardous_C",
    74: "Cargo_hazardous_D",
    79: "Cargo_unspecified",
    80: "Tanker",
    range(81, 85): "Tanker_hazardous_types_A_to_D",
    89: "Tanker_unspecified",
    90: "Other_Special_category",
    range(91, 93): "Reserved_for_future_use",
    99: "Undefined_Unknown"
}

# === 3️⃣ Helper: get vessel name from code ===
def get_vessel_name(code):
    try:
        code = int(float(code))
        for k, v in vessel_type_map.items():
            if isinstance(k, range) and code in k:
                return v
            elif k == code:
                return v
    except:
        pass
    return f"Unknown_{code}"

# === 4️⃣ Rename columns dynamically ===
def rename_vessel_columns(df):
    new_cols = {}
    for col in df.columns:
        match = re.search(r'vessel_type_(\d+\.?\d*)', col)
        if match:
            code = match.group(1)
            new_cols[col] = get_vessel_name(code)
        else:
            new_cols[col] = col
    return df.rename(columns=new_cols)

# === 5️⃣ Apply renaming and save ===
print("Loading file...")
df = pd.read_csv(INPUT_CSV)
print(f"Original columns: {len(df.columns)}")

df_renamed = rename_vessel_columns(df)
df_renamed.to_csv(OUTPUT_CSV, index=False)

print(f"Renamed file saved to:\n{OUTPUT_CSV}")
print(f"Updated columns ({len(df_renamed.columns)}):")
for col in df_renamed.columns:
    if "vessel" in col or "Cargo" in col or "Tanker" in col:
        print(" -", col)


Loading file...
Original columns: 36
Renamed file saved to:
/content/drive/MyDrive/NASA_SPACE_APP/oil_points_AIS_stats_renamed_18_19_20_.csv
Updated columns (36):
 - Cargo
 - Cargo_unspecified
 - Tanker
 - Cargo_hazardous_A
 - Search_and_rescue_vessel
 - Cargo_hazardous_B
 - Tanker_hazardous_types_A_to_D
 - Law_enforcement_vessel
 - Cargo_hazardous_C
 - Tanker_hazardous_types_A_to_D


In [ ]:
import pandas as pd
import glob

# Path to your CSV files (adjust to your setup)
files = glob.glob("/content/drive/MyDrive/NASA_SPACE_APP/oil_points_AIS_stats_renamed_*.csv")

dfs = [pd.read_csv(f) for f in files]

# Append all rows together
merged_df = pd.concat(dfs, axis=0, ignore_index=True)

print("Merged shape:", merged_df.shape)
merged_df.head()

# Save final combined dataset
merged_df.to_csv("/content/drive/MyDrive/NASA_SPACE_APP/oil_points_AIS_stats_all.csv", index=False)


Merged shape: (5887, 55)


In [ ]:
import pandas as pd
import glob

# Path to your CSV files (adjust to your setup)
files = glob.glob("/content/drive/MyDrive/NASA_SPACE_APP/AIS_daily_stats_*.csv")

dfs = [pd.read_csv(f) for f in files]

# Append all rows together
merged_df = pd.concat(dfs, axis=0, ignore_index=True)

print("Merged shape:", merged_df.shape)
merged_df.head()

# Save final combined dataset
merged_df.to_csv("/content/drive/MyDrive/NASA_SPACE_APP/AIS_daily_stats_all.csv", index=False)


Merged shape: (461, 8)


In [ ]:
import pandas as pd

# ---- Load SAR + NASA POWER (oil candidates only) ----
sar_df = pd.read_csv("/content/drive/MyDrive/NASA_SPACE_APP/SAR_with_NASA_POWER_full.csv")
sar_df['date'] = pd.to_datetime(sar_df['date'])
sar_df = sar_df[sar_df['oil_candidate'] == 1]  # Only oil candidate points

# ---- Load AIS oil points ----
ais_df = pd.read_csv("/content/drive/MyDrive/NASA_SPACE_APP/oil_points_AIS_stats_all.csv")
ais_df['Date'] = pd.to_datetime(ais_df['Date'])
ais_df.rename(columns={'Date':'date','oil_lat':'latitude','oil_lon':'longitude'}, inplace=True)

# ---- Keep only SAR points for dates available in AIS ----
available_dates = ais_df['date'].unique()
sar_df = sar_df[sar_df['date'].isin(available_dates)]

# ---- Merge SAR + AIS on date, latitude, longitude ----
merged_df = pd.merge(
    sar_df,
    ais_df,
    on=['date','latitude','longitude'],
    how='left'
)

# ---- Save merged dataframe ----
merged_df.to_csv("/content/drive/MyDrive/NASA_SPACE_APP/SAR_envi_oil_with_AIS.csv", index=False)
print("Merged SAR + AIS data saved!")


Merged SAR + AIS data saved!
